In [103]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from pathlib import Path 
import pandas as pd
import numpy as np
import ast
import wikipediaapi
from pathlib import Path
import time
from time import sleep

import csv
import requests
from bs4 import BeautifulSoup
import urllib.request, urllib.parse, urllib.error
import ssl

In [5]:
path = Path('./the_movie_dataset')
files = [p for p in path.glob('*.csv')]
files
credits = pd.read_csv([f for f in files if 'credits' in str(f)][0], dtype={'id':'str'})
credits.rename({'id':'tmdbId'},axis=1, inplace=True)
print(credits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   tmdbId  45476 non-null  object
dtypes: object(3)
memory usage: 1.0+ MB
None


#### credits 파일에서 main actor 3명을 뽑고, unique_actor의 이름으로 dataFrame을 만들어두었음

In [6]:
credits['cast_main_5'] = ( credits['cast'].apply(ast.literal_eval)
                            .apply(lambda x: [ x['name'] for x in x[:3]] ) )
credits['cast_main_5_gender'] = ( credits['cast'].apply(ast.literal_eval)
                            .apply(lambda x: [ x['gender'] for x in x[:3]] ) )

credits[['actor_1_name',
         'actor_2_name',
         'actor_3_name']] = ( credits['cast_main_5']
                             .apply(lambda x: pd.Series(x)) )


credits[['actor_1_gender',
         'actor_2_gender',
         'actor_3_gender']] = ( credits['cast_main_5_gender']
                             .apply(lambda x: pd.Series(x)) )

In [74]:
unique_actors = set([item for sublist in credits['cast_main_5'].to_list() for item in sublist])
# (0,0)을 tuple로 갖는 dictionary
unique_actors = dict.fromkeys(unique_actors, (0,0)) 
list(unique_actors)[:5]

['Aki Avni',
 'Coen van Vrijberghe de Coningh',
 'Jim Driscoll',
 'Raivo E. Tamm',
 'Rasmus Bjerg']

In [75]:
# unique_actore의 수
len(unique_actors)

47481

In [6]:
unique_actors

{'Jarlath Conroy': (0, 0),
 'Colin Moy': (0, 0),
 'Chantal Barré': (0, 0),
 'Kenny Wormald': (0, 0),
 'Joanna David': (0, 0),
 'Terence De Marney': (0, 0),
 'Marilyn Bautista': (0, 0),
 'Dorota Stalińska': (0, 0),
 'Jocelyn Towne': (0, 0),
 'Si Newhouse IV': (0, 0),
 'Kichiemon Nakamura': (0, 0),
 'Abdul Salis': (0, 0),
 'Chloe Levine': (0, 0),
 'Nada Abou Farhat': (0, 0),
 'Ray Valenti': (0, 0),
 'Earl Hindman': (0, 0),
 'Lynn Monteil': (0, 0),
 'Tasha Smith': (0, 0),
 'Natasha Morley': (0, 0),
 'Gabriella Pession': (0, 0),
 'Park Hyuk-kwon': (0, 0),
 'William Alva': (0, 0),
 'Josh Groban': (0, 0),
 'John Gordon Sinclair': (0, 0),
 'Madonna Sebastian': (0, 0),
 'Ansiba Hassan': (0, 0),
 'Kitty Aldridge': (0, 0),
 'John Benjamin Martin': (0, 0),
 'Misa Jokisalo': (0, 0),
 'Riccardo Billi': (0, 0),
 'Scott Beehner': (0, 0),
 'Krsna Angulo': (0, 0),
 'Jose Yenque': (0, 0),
 'Matty Liu': (0, 0),
 'Phil Vassar': (0, 0),
 'Kiana Cason': (0, 0),
 'Richard Norton': (0, 0),
 'Zvezda Angelovska

In [76]:
df = pd.DataFrame(unique_actors).T.rename({0:'won',1:'nominat'},axis=1).reset_index()

In [77]:
df.to_csv('actors.csv')

In [82]:
df

,index,won,nominat
0,Aki Avni,0,0
1,Coen van Vrijberghe de Coningh,0,0
2,Jim Driscoll,0,0
3,Raivo E. Tamm,0,0
4,Rasmus Bjerg,0,0
...,...,...,...
47476,Eddie Mills,0,0
47477,Sirisin Siripornsmathikul,0,0
47478,Marleen Lohse,0,0
47479,Silvio Hernández,0,0


#### 속도가 너무 느리기 때문에 파일을 3개로 나눠서 동시에 수행

In [99]:
paralle = np.linspace(0,df.shape[0],4)
paralle

array([    0., 15827., 31654., 47481.])

In [101]:
df_sub = df.iloc[:int(paralle[1])]
df_sub

,index,won,nominat
0,Aki Avni,0,0
1,Coen van Vrijberghe de Coningh,0,0
2,Jim Driscoll,0,0
3,Raivo E. Tamm,0,0
4,Rasmus Bjerg,0,0
...,...,...,...
15822,Paul Butcher,0,0
15823,Roger Gobeth,0,0
15824,Marcus Hearn,0,0
15825,Antonella Ríos,0,0


#### url 규칙 
- {name}자리에 배우 이름의 빈칸이 _로 채워서 들어간다 https://en.wikipedia.org/wiki/{name}
- 유명 배우들은 List_of_awards_and_nominations_received_by_{name} 으로 패이지가 따로 분류 되어있다.
#### tag 규칙
- 어느 tag에 won, nominate가 들어가있을지 예상 규칙을 찾을 수 없기 때문에 통으로 긁어서 won, noimnat text의 수를 count하였다.

In [104]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

stats = time.time()
exceptions = 'List_of_awards_and_nominations_received_by_'
with open('actor_awards_1.csv','w') as csvfile:
    fieldnames = ['name','won','nominat']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()


    for i,n in enumerate(df_sub['index']):
        
        # print progress 1000단위로 진행 되고 있는지 print
        _, r = divmod(i,1000) 
        if r == 0:
            print(i,df_sub.shape[0])
        
        name = n.replace(' ','_')
        sleep(np.round(np.random.random_sample(),3))
        try : # 수상 리스트가 따로 있을 때
            url = f'https://en.wikipedia.org/wiki/{exceptions}{name}'
            html = urllib.request.urlopen(url, context=ctx).read()
        except :

            try: # 따로 없을 때, 이름으로 찾기
                url = f'https://en.wikipedia.org/wiki/{name}'
                html = urllib.request.urlopen(url, context=ctx).read()
            except: # 이름이 없을 때는 방법 없음
                cnt_won = np.nan
                cnt_nomi = np.nan
                continue
                
        # html parser 
        soup = BeautifulSoup(html, 'html.parser')
        t =soup.find(id='bodyContent').text
        cnt_won = t.lower().count('won')
        cnt_nomi = t.lower().count('nominat')
        
        writer.writerow({'name':n
                          ,'won':cnt_won
                          ,'nominat':cnt_nomi
                         })
print(time.time()-stats)

0 15827
1000 15827
2000 15827
3000 15827
4000 15827
5000 15827
6000 15827
7000 15827
8000 15827
9000 15827
10000 15827
11000 15827
12000 15827
13000 15827
14000 15827
15000 15827
39653.230185985565
